In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf

import sys
code_dir = '/cluster/home/kheuto01/code/zero-inflated-gp/'
sys.path.append(code_dir)

import os
import subprocess
import pandas as pd
import numpy as np

2023-03-01 11:29:16.580999: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-01 11:29:16.703409: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-01 11:29:16.709065: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-01 11:29:16.709079: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [16]:
times = ['annual', 'quarter','semi']
locs = ['tract','town','group']
start_years = [ 2000,2010]
covs = [ '-auto','-auto-svi','all']
models = ['normal', 'poisson']
learning_rates = [0.1,0.01,0.001]
inducing_points = [400,200]

In [3]:

data_dir='/cluster/tufts/hugheslab/datasets/NSF_OD/results_20220606_update/'

log_dir='/cluster/tufts/hugheslab/kheuto01/opioid/logs/converge'


In [28]:
counter = 0
for time in times:

    for loc in locs:
        for start_year in start_years:
            for cov in covs:
                for model in models:
                    for num_inducing in inducing_points:


                        arg_parts = [
                            f"--data_dir {data_dir}",
                            f"--log_dir {log_dir}",
                            f"--time {time}",
                            f"--loc {loc}",
                            f"--start_year {start_year}",
                            f"--cov='{cov}'",
                            f"--test_years 2",
                            f"--timestep_col timestep",
                            f"--geography_col geoid",
                            f"--model {model}",
                            f"--num_inducing {num_inducing}",
                            f"--learning_rates {' '.join([str(lr) for lr in learning_rates])}",
                        ]



                        arg_cmd = ' '.join(arg_parts)
                        command = (f"args='{arg_cmd}' sbatch < /cluster/home/kheuto01/code/opioid-overdose-models/zinf_gp/check_unc.slurm")
                        
                        counter += 1
                        subprocess.run(command, shell=True, check=True)
                        
    

Submitted batch job 41899774
Submitted batch job 41899775
Submitted batch job 41899776
Submitted batch job 41899777
Submitted batch job 41899778
Submitted batch job 41899779
Submitted batch job 41899780
Submitted batch job 41899781
Submitted batch job 41899782
Submitted batch job 41899783
Submitted batch job 41899784
Submitted batch job 41899785
Submitted batch job 41899786
Submitted batch job 41899787
Submitted batch job 41899788
Submitted batch job 41899789
Submitted batch job 41899790
Submitted batch job 41899791
Submitted batch job 41899792
Submitted batch job 41899793
Submitted batch job 41899794
Submitted batch job 41899795
Submitted batch job 41899796
Submitted batch job 41899797
Submitted batch job 41899798
Submitted batch job 41899799
Submitted batch job 41899800
Submitted batch job 41899801
Submitted batch job 41899802
Submitted batch job 41899803
Submitted batch job 41899804
Submitted batch job 41899805
Submitted batch job 41899806
Submitted batch job 41899807
Submitted batc

In [24]:
command

"args='--data_dir /cluster/tufts/hugheslab/datasets/NSF_OD/results_20220606_update/ --log_dir /cluster/tufts/hugheslab/kheuto01/opioid/logs/converge --time semi --loc group --start_year 2010 --cov='all' --test_years 2 --timestep_col timestep --geography_col geoid --model poisson --num_inducing 200 --learning_rates 0.1 0.01 0.001' sbatch < /cluster/home/kheuto01/code/opioid-overdose-models/zinf_gp/check_unc.slurm"

In [25]:
subprocess.run(command, shell=True, check=True)

Submitted batch job 41899341


CompletedProcess(args="args='--data_dir /cluster/tufts/hugheslab/datasets/NSF_OD/results_20220606_update/ --log_dir /cluster/tufts/hugheslab/kheuto01/opioid/logs/converge --time semi --loc group --start_year 2010 --cov='all' --test_years 2 --timestep_col timestep --geography_col geoid --model poisson --num_inducing 200 --learning_rates 0.1 0.01 0.001' sbatch < /cluster/home/kheuto01/code/opioid-overdose-models/zinf_gp/check_unc.slurm", returncode=0)

In [73]:
all_results = []
counter = 0
for time in times:

    for loc in locs:
        for start_year in start_years:
            for cov in covs:
                for model in models:
                    for num_inducing in inducing_points:

                        unc_file = os.path.join(log_dir, f'unc_results_{time}_{loc}_{model}_{start_year}_{cov}_{num_inducing}.csv')

                        results = pd.read_csv(unc_file,index_col=0)
                        
                        low_2019, med_2019, high_2019 = results.loc[0,:]
                        low_2020, med_2020, high_2020 = results.loc[1,:]
                        
                        data = [time, loc, start_year, cov, model, num_inducing, low_2019, med_2019, high_2019, low_2020, med_2020, high_2020]
                        
                        all_results.append(data)
                        
df = pd.DataFrame(all_results,columns=['time', 'loc', 'start', 'covs', 'model', 'num_inducing', 'low_2019', 'mean_2019', 'high_2019', 'low_2020', 'mean_2020', 'high_2020'])

In [75]:
df.to_csv('~/tract_uncertainty_results.csv')

In [66]:
a

0.2584754006405382

In [67]:
b

0.2775113440448404

In [68]:
c

0.2934346846261491